In [1]:
# ==========================================
# CELL 1: INSTALL & SETUP
# ==========================================
# !pip install rank_bm25 sentence-transformers faiss-cpu

import os
import pickle
import numpy as np
import pandas as pd
import faiss
from groq import Groq
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder
from tqdm import tqdm
import gc
import string
import json
from rag_evaluation import RAGEvaluator

# Load Environment Variables
from dotenv import load_dotenv
load_dotenv(override=True)

# Define Model Constants
EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"
# LLM_MODEL = "llama-3.3-70b-versatile"
LLM_MODEL = "openai/gpt-oss-120b"

# Initialize Groq Client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

print("✅ Advanced RAG Libraries Loaded.")

✅ Advanced RAG Libraries Loaded.


In [2]:
# 2. DEFINE MODEL & PATHS
RERANK_MODEL_ID = "cross-encoder/ms-marco-MiniLM-L-6-v2" 
LOCAL_RERANK_PATH = "./models/rerank_model"

print(f"⏳ Checking for Re-Ranker at: {LOCAL_RERANK_PATH}...")

# 3. LOAD OR DOWNLOAD LOGIC
if os.path.exists(LOCAL_RERANK_PATH):
    print("   • Found local model cache. Loading from disk...")
    try:
        # Load from local folder to avoid internet dependency
        cross_encoder = CrossEncoder(LOCAL_RERANK_PATH, automodel_args={"local_files_only": True})
        print("✅ Re-Ranker loaded from local cache.")
    except Exception as e:
        print(f"⚠️ Local load failed ({e}). Re-downloading...")
        cross_encoder = CrossEncoder(RERANK_MODEL_ID)
        cross_encoder.save(LOCAL_RERANK_PATH)
        print("✅ Re-Ranker downloaded and repaired.")
else:
    print(f"   • Local cache not found. Downloading {RERANK_MODEL_ID}...")
    # Download from Hub
    cross_encoder = CrossEncoder(RERANK_MODEL_ID)
    # Save to local folder for next time
    print(f"   • Saving model to {LOCAL_RERANK_PATH}...")
    cross_encoder.save(LOCAL_RERANK_PATH)
    print("✅ Re-Ranker downloaded and saved.")

The CrossEncoder `automodel_args` argument was renamed and is now deprecated, please use `model_kwargs` instead.


⏳ Checking for Re-Ranker at: ./models/rerank_model...
   • Found local model cache. Loading from disk...
⚠️ Local load failed (sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder._load_model() got multiple values for keyword argument 'local_files_only'). Re-downloading...
✅ Re-Ranker downloaded and repaired.


In [3]:
import psutil
import os

def get_memory_usage():
    # 1. Overall System Memory
    system_mem = psutil.virtual_memory()
    print(f"System Total:     {system_mem.total / (1024**3):.2f} GB")
    print(f"System Available: {system_mem.available / (1024**3):.2f} GB")
    print(f"System Used:      {system_mem.percent}%")
    
    # 2. Specific Process Memory (This Notebook)
    process = psutil.Process(os.getpid())
    process_mem = process.memory_info().rss  # RSS = Resident Set Size (actual RAM used)
    print(f"Notebook Usage:   {process_mem / (1024**3):.2f} GB")

get_memory_usage()

System Total:     24.00 GB
System Available: 10.19 GB
System Used:      57.5%
Notebook Usage:   0.63 GB


In [4]:
# ==========================================
# CELL 2: THE ADVANCED RETRIEVER (Hybrid)
# ==========================================

class AdvancedRetriever:
    def __init__(self, model_path="./models"):
        print("⚙️ Initializing Advanced Retrieval Engine...")
        os.makedirs(model_path, exist_ok=True)
        
        # --- 1. Load Embedding Model (Locally Cached) ---
        embed_model_path = f"{model_path}/embedding_model"
        
        if os.path.exists(embed_model_path):
            print(f"   • Loading Embedding Model from local cache: {embed_model_path}...")
            self.encoder = SentenceTransformer(embed_model_path)
        else:
            print(f"   • Downloading Embedding Model ({EMBED_MODEL_ID})...")
            self.encoder = SentenceTransformer(EMBED_MODEL_ID)
            print(f"   • Saving model to {embed_model_path} for future runs...")
            self.encoder.save(embed_model_path)
        
        # --- 2. Load FAISS Index (Dense) ---
        self.index = faiss.read_index(f"{model_path}/faiss_index.bin")
        
        # --- 3. Load Metadata (Content) ---
        with open(f"{model_path}/chunk_metadata.pkl", "rb") as f:
            self.chunks = pickle.load(f)
            
        print(f"   • Loaded {len(self.chunks)} documents from FAISS.")
        
        # --- 4. Load OR Build BM25 Index (Sparse) ---
        bm25_path = f"{model_path}/bm25_index.pkl"
        
        if os.path.exists(bm25_path):
            print(f"   • Found cached BM25 Index at {bm25_path}. Loading...")
            with open(bm25_path, "rb") as f:
                self.bm25 = pickle.load(f)
        else:
            print("   • No cache found. Building BM25 Index from scratch...")
            tokenized_corpus = [doc['text'].lower().split() for doc in self.chunks]
            self.bm25 = BM25Okapi(tokenized_corpus)
            
            print(f"   • Saving BM25 Index to {bm25_path}...")
            with open(bm25_path, "wb") as f:
                pickle.dump(self.bm25, f)
                
        print("✅ Hybrid System Ready (Dense + Sparse).")

    def search_dense(self, query, k=50):
        """Standard Vector Search"""
        # BGE requires instruction for queries
        query_prompt = f"Represent this sentence for searching relevant passages: {query}"
        query_vec = self.encoder.encode([query_prompt], normalize_embeddings=True, convert_to_numpy=True)
        
        D, I = self.index.search(query_vec, k)
        
        results = []
        for i in range(k):
            idx = I[0][i]
            # We don't trust vector scores blindly anymore, so we just store the rank
            results.append(self.chunks[idx])
            
        return results

    def search_sparse(self, query, k=50):
        """Keyword Search using BM25"""
        tokenized_query = query.lower().split()
        # Get top-k documents
        docs = self.bm25.get_top_n(tokenized_query, self.chunks, n=k)
        return docs

    def hybrid_search(self, query, k=50, rrf_k=60):
        """
        Performs Reciprocal Rank Fusion (RRF) on Dense and Sparse results.
        Returns the top-k fused results.
        """
        # 1. Run both searches in parallel (conceptually)
        dense_results = self.search_dense(query, k=k)
        sparse_results = self.search_sparse(query, k=k)
        
        # 2. Score Map: {chunk_id: rrf_score}
        # We use the text or title+text as a unique key since we don't have explicit IDs in metadata yet
        # (Ideally, add a UUID to chunks in Phase 1, but using text hash works for now)
        scores = {}
        
        # Helper to get unique key
        def get_key(doc):
            return hash(doc['text']) 

        # Map objects for easy retrieval later
        doc_map = {}

        # 3. Calculate RRF for Dense
        for rank, doc in enumerate(dense_results):
            doc_id = get_key(doc)
            doc_map[doc_id] = doc
            if doc_id not in scores: scores[doc_id] = 0
            # RRF Formula: 1 / (k + rank)
            scores[doc_id] += 1 / (rrf_k + rank + 1) # +1 because rank is 0-indexed

        # 4. Calculate RRF for Sparse
        for rank, doc in enumerate(sparse_results):
            doc_id = get_key(doc)
            doc_map[doc_id] = doc
            if doc_id not in scores: scores[doc_id] = 0
            scores[doc_id] += 1 / (rrf_k + rank + 1)
            
        # 5. Sort by Combined Score
        sorted_ids = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)
        
        # 6. Return Top K Docs
        fused_results = [doc_map[uid] for uid in sorted_ids[:k]]
        
        return fused_results

# Initialize (This might take 30-60s to build BM25)
hybrid_retriever = AdvancedRetriever()

⚙️ Initializing Advanced Retrieval Engine...
   • Loading Embedding Model from local cache: ./models/embedding_model...
   • Loaded 39141 documents from FAISS.
   • Found cached BM25 Index at ./models/bm25_index.pkl. Loading...
✅ Hybrid System Ready (Dense + Sparse).


In [5]:
# ==========================================
# CELL 3: TEST HYBRID RETRIEVAL
# ==========================================

test_query = "What is error 404?" 
# Or use a specific name from your dataset like "Mathew Knowles"

results = hybrid_retriever.hybrid_search(test_query, k=5)

print(f"🔎 Results for: '{test_query}'")
for i, res in enumerate(results):
    print(f"[{i+1}] {res['title']}")
    print(f"    Preview: {res['text'][:150]}...")

🔎 Results for: 'What is error 404?'
[1] Translation
    Preview: Translation is the communication of the meaning of a source-language text by means of an equivalent target-language text. The English language draws a...
[2] Xbox_360
    Preview: To aid customers with defective consoles, Microsoft extended the Xbox 360's manufacturer's warranty to three years for hardware failure problems that ...
[3] Wayback_Machine
    Preview: Limitations
In 2014, there was a six-month lag time between when a website was crawled and when it became available for viewing in the Wayback Machine...
[4] Xbox_360
    Preview: Timeline
United States
Technical problems
The original model of the Xbox 360 has been subject to a number of technical problems. Since the console's r...
[5] Compact_disc
    Preview: Error scanning can reliably predict data losses caused by media deterioration. Support of error scanning differs between vendors and models of optical...


In [6]:
# ==========================================
# CELL 4: INITIALIZE RE-RANKER
# ==========================================
def rerank_results(query, initial_results, top_k=5):
    """
    Takes a list of 'candidate' chunks (e.g., top 50 from Hybrid Search),
    scores them with the Cross-Encoder, and returns the top K best ones.
    """
    if not initial_results:
        return []
    
    # 1. Prepare Pairs: [[Query, Text1], [Query, Text2], ...]
    # The model expects a list of pairs to evaluate.
    pairs = [[query, doc['text']] for doc in initial_results]
    
    # 2. Score Pairs
    # This is the "expensive" step. It reads every word.
    scores = cross_encoder.predict(pairs)
    
    # 3. Attach Scores to Documents
    ranked_results = []
    for i, doc in enumerate(initial_results):
        # Create a copy so we don't mess up the original list
        doc_copy = doc.copy()
        doc_copy['rerank_score'] = float(scores[i])
        ranked_results.append(doc_copy)
    
    # 4. Sort by Score (High to Low)
    ranked_results.sort(key=lambda x: x['rerank_score'], reverse=True)
    
    # 5. Return Top K
    return ranked_results[:top_k]

In [7]:
# ==========================================
# CELL 5: THE ADVANCED PIPELINE
# ==========================================
LLM_MODEL = "llama-3.3-70b-versatile"
def generate_advanced_rag_answer(query, retriever_instance, top_k_retrieval=50, top_k_final=5, llm_model=LLM_MODEL):
    """
    The Advanced RAG Pipeline:
    1. Hybrid Retrieval (Vectors + Keywords) -> Get Top 50
    2. Re-Ranking (Cross-Encoder) -> Filter to Top 5
    3. Generation (LLM) -> Answer
    """
    
    # --- PHASE 1: RETRIEVAL (Wide Net) ---
    # We fetch more docs (50) to ensure we didn't miss anything.
    candidates = retriever_instance.hybrid_search(query, k=top_k_retrieval)
    
    # --- PHASE 2: RE-RANKING (Precision Filter) ---
    # We let the Cross-Encoder pick the absolute best ones.
    final_docs = rerank_results(query, candidates, top_k=top_k_final)
    
    # --- PHASE 3: CONTEXT CONSTRUCTION ---
    context_text = ""
    for i, doc in enumerate(final_docs):
        context_text += f"Source {i+1} [Score: {doc['rerank_score']:.4f}]:\n{doc['text']}\n\n"
    
    # --- PHASE 4: GENERATION ---
    system_prompt = """You are a precise and helpful AI assistant.
    Use the provided Context Information to answer the user's question.
    If the answer is not present in the context, strictly state: "I cannot answer this based on the provided documents."
    Do not hallucinate."""
    
    user_prompt = f"""
    Context Information:
    ---------------------
    {context_text}
    ---------------------
    
    User Question: {query}
    """
    
    try:
        completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            model=LLM_MODEL,
            temperature=0.1,
            max_tokens=512
        )
        answer = completion.choices[0].message.content
    except Exception as e:
        answer = f"Error generating answer: {e}"
        
    return answer, final_docs

In [8]:
# ==========================================
# CELL 6: ONE-OFF INTERACTIVE TEST
# ==========================================
# Choose a query that requires specific facts to test the Re-Ranker's precision
# query = "When did Beyonce start becoming popular?"
# Alternative tough queries to try later:
query = "What is the exact height of the Eiffel Tower?"  # Tests number precision
# query = "Who managed Destiny's Child?" #(Tests entity relation)

print(f"🚀 Running Advanced RAG Pipeline for: '{query}'")
print(f"   • Strategy: Hybrid Retrieval (Top 50) -> Cross-Encoder Re-Ranking (Top 5)")

# 1. Execute Pipeline
answer, final_docs = generate_advanced_rag_answer(
    query, 
    hybrid_retriever,    # Your AdvancedRetriever instance from Cell 2
    top_k_retrieval=50,  # Wide Net
    top_k_final=5        # Precision Filter
)

# 2. Display Final Output
print("\n🤖 Generated Answer:")
print("=" * 60)
print(answer)
print("=" * 60)

# 3. Inspect the "Why" (The Re-Ranker's decisions)
print("\n🔍 Top 5 Re-Ranked Contexts (Sorted by Relevance Score):")
for i, doc in enumerate(final_docs):
    print(f"[{i+1}] Score: {doc['rerank_score']:.4f} | Title: {doc['title']}")
    print(f"    Preview: {doc['text'][:200]}...")
    print("-" * 40)

# 4. Optional: Check if we filtered out noise
# If the scores drop off sharply (e.g., 0.98 -> 0.10), the Re-Ranker is working well.
print(f"\n✅ Pipeline Test Complete.")

🚀 Running Advanced RAG Pipeline for: 'What is the exact height of the Eiffel Tower?'
   • Strategy: Hybrid Retrieval (Top 50) -> Cross-Encoder Re-Ranking (Top 5)

🤖 Generated Answer:
I cannot answer this based on the provided documents.

🔍 Top 5 Re-Ranked Contexts (Sorted by Relevance Score):
[1] Score: 2.7520 | Title: Elevator
    Preview: Eiffel Tower
The Eiffel Tower has Otis double-deck elevators built into the legs of the tower, serving the ground level to the first and second levels. Even though the shaft runs diagonally upwards wi...
----------------------------------------
[2] Score: 2.3572 | Title: Paris
    Preview: Paris' urbanism laws have been under strict control since the early 17th century, particularly where street-front alignment, building height and building distribution is concerned. In recent developme...
----------------------------------------
[3] Score: -1.1881 | Title: Paris
    Preview: Paris's urbanism laws have been under strict control since the early 17th 

In [9]:
# ==========================================
# CELL 7: INITIALIZE EVALUATOR
# ==========================================
# Initialize with the ADVANCED components
evaluator = RAGEvaluator(
    retriever_instance=hybrid_retriever,        # Use the NEW Hybrid Retriever
    generator_func=generate_advanced_rag_answer, # Use the NEW Re-Ranking Pipeline
    groq_client=client
)

print("✅ Advanced Evaluator Initialized.")

✅ Advanced Evaluator Initialized.


## WITHOUT IMPOSSIBLE Questions

In [ ]:
### WITHOUT IMPOSSIBLE
with open("./data/raw/squad_eval_set_all.json", "r") as f:
    raw_data = json.load(f)
    
# Filter only validation data
# eval_dataset = raw_data
eval_dataset = [x for x in raw_data if x['is_impossible'] == False]
# eval_dataset = [x for x in raw_data if x['split'] == 'validation']
print(f"📚 Loaded {len(eval_dataset)} validation questions.")

del raw_data
gc.collect()

# ==========================================
# CELL 3: RUN EXPERIMENT 1 (Baseline)
# ==========================================

# Let's run a test with 30 questions
df, summary = evaluator.run_experiment(
    dataset=eval_dataset,
    experiment_name="not_impossible",
    rag_type="advanced_rag",
    model_name="llama-3.3-70b-versatile",
    # model_name="openai/gpt-oss-120b",
    sample_size=50,
    target_rpm=15,                     # Slower for safety (Judge adds calls)
    use_llm_judge=False,                # Enable Judge
    # judge_model = "llama-3.3-70b-versatile"
    # judge_model="openai/gpt-oss-20b"
    judge_model="openai/gpt-oss-120b" # Pick your judge!
)

# View the first few rows of the saved data
df.head()

In [ ]:
df_not_impossible = pd.read_csv("")
# df_not_impossible.to_csv("final_results/NI_adv_rag.csv")
df_not_impossible[df_not_impossible['generated_answer'].str.contains('Error')]['generated_answer']

30    Error generating answer: Error code: 429 - {'e...
31    Error generating answer: Error code: 429 - {'e...
32    Error generating answer: Error code: 429 - {'e...
33    Error generating answer: Error code: 429 - {'e...
34    Error generating answer: Error code: 429 - {'e...
35    Error generating answer: Error code: 429 - {'e...
36    Error generating answer: Error code: 429 - {'e...
37    Error generating answer: Error code: 429 - {'e...
38    Error generating answer: Error code: 429 - {'e...
39    Error generating answer: Error code: 429 - {'e...
40    Error generating answer: Error code: 429 - {'e...
41    Error generating answer: Error code: 429 - {'e...
42    Error generating answer: Error code: 429 - {'e...
43    Error generating answer: Error code: 429 - {'e...
44    Error generating answer: Error code: 429 - {'e...
45    Error generating answer: Error code: 429 - {'e...
46    Error generating answer: Error code: 429 - {'e...
47    Error generating answer: Error code: 429 -

In [ ]:
csv_path = "final_results/NI_adv_rag.csv"
df = pd.read_csv(csv_path)
print(df.shape)
df[df['generated_answer'].str.contains('Error')]['generated_answer']

start = 0
end = 10
df_batch = df[start:end]
print(df_batch.shape)
df_batch.tail()

In [ ]:
### LLM AS A JUDGE
df_batch = df[start:end]
print(df_batch.shape) ### RUN AGAIN

# Run the judge
df_judged, summary_judged = evaluator.evaluate_batch(
    results_path_or_df=df_batch,
    judge_model="openai/gpt-oss-120b",
    target_rpm = 15
)
for k, v in summary_judged.items(): print(f"{k:<25}: {v:.4f}")

In [ ]:
df0_10 = pd.read_csv('')
df0_10[df0_10['judge_reasoning'].str.contains('Error')]['judge_reasoning']

In [ ]:
### WITHOUT IMPOSSIBLE
df0_10 = pd.read_csv('')
df10_20 = pd.read_csv('')
df20_30 = pd.read_csv('')
df30_40 = pd.read_csv('')
df40_50 = pd.read_csv('')
df_final_NI = pd.concat([df0_10, df10_20, df20_30, df30_40, df40_50], ignore_index=True)
df_final_NI.to_csv('final_results/NI_adv_rag_judged.csv', index=False)

## WITH IMPOSSIBLE 

In [ ]:
### WITH IMPOSSIBLE
with open("./data/raw/squad_eval_set_all.json", "r") as f:
    raw_data = json.load(f)
    
# Filter only validation data
eval_dataset = raw_data
# eval_dataset = [x for x in raw_data if x['is_impossible'] == False]
# eval_dataset = [x for x in raw_data if x['split'] == 'validation']
print(f"📚 Loaded {len(eval_dataset)} validation questions.")

del raw_data
gc.collect()

# ==========================================
# CELL 3: RUN EXPERIMENT 1 (Baseline)
# ==========================================

df1, summary = evaluator.run_experiment(
    dataset=eval_dataset,
    experiment_name="with_impossible",
    rag_type="advanced_rag",
    model_name="llama-3.3-70b-versatile",
    # model_name="openai/gpt-oss-120b",
    sample_size=50,
    target_rpm=15,                     # Slower for safety (Judge adds calls)
    use_llm_judge=False,                # Enable Judge
    # judge_model = "llama-3.3-70b-versatile"
    # judge_model="openai/gpt-oss-20b"
    judge_model="openai/gpt-oss-120b" 
)

# View the first few rows of the saved data
df1.head()

In [ ]:
df_impossible = pd.read_csv("")
df_impossible.to_csv("final_results/Imp_adv_rag.csv")
df_impossible[df_impossible['generated_answer'].str.contains('Error')]['generated_answer']

In [ ]:
csv_path = "final_results/Imp_adv_rag.csv"
df = pd.read_csv(csv_path)
print(df.shape)
df[df['generated_answer'].str.contains('Error')]['generated_answer']

start = 0
end = 10
df_batch = df[start:end]
print(df_batch.shape)
df_batch.tail()

In [ ]:
### LLM AS A JUDGE
df_batch = df[start:end]
print(df_batch.shape) ### RUN AGAIN

# Run the judge
df_judged, summary_judged = evaluator.evaluate_batch(
    results_path_or_df=df_batch,
    judge_model="openai/gpt-oss-120b",
    target_rpm = 15
)
for k, v in summary_judged.items(): print(f"{k:<25}: {v:.4f}")

In [ ]:
df0_10 = pd.read_csv('')
df0_10[df0_10['judge_reasoning'].str.contains('Error')]['judge_reasoning']

In [ ]:
### WITH IMPOSSIBLE
df0_10 = pd.read_csv('')
df10_20 = pd.read_csv('')
df20_30 = pd.read_csv('')
df30_40 = pd.read_csv('')
df40_50 = pd.read_csv('')
df_final_Imp = pd.concat([df0_10, df10_20, df20_30, df30_40, df40_50], ignore_index=True)
df_final_Imp.to_csv('final_results/Imp_adv_rag_judged.csv', index=False)

## COMBINING BOTH FILES

In [ ]:
df_final_NI = pd.read_csv('final_results/NI_adv_rag_judged.csv')
df_final_Imp = pd.read_csv('final_results/Imp_adv_rag_judged.csv')
print(df_final_NI.shape)
print(df_final_Imp.shape)
df_final = pd.concat([df_final_NI, df_final_Imp], ignore_index=True)
print(df_final.shape)
df_final.to_csv('final_results/adv_rag_judged.csv', index=False)